## 2.1 An Introduction to fugashi :en

## 2.1 fugashi の紹介 :ja

In this section you'll learn how to do Japanese tokenization using fugashi, a MeCab wrapper, and the unidic-lite dictionary. :en

この章では、日本語の形態素解析器 MeCab のラッパーである fugashi と、辞書の unidic-lite を使い、日本語の形態素解析の基礎について学びます。 :ja


|surface|pos1|pos2|pos3|lemma|pron|kana|goshu|
|-------|----|----|----|-----|----|----|-----|
|喫茶|名詞|普通名詞|一般|喫茶|キッサ|キッサ|漢|
|店|接尾辞|名詞的|一般|店|テン|テン|漢|
|と|助詞|格助詞|*|と|ト|ト|和|
|カフェ|名詞|普通名詞|一般|カフェ-cafe|カフェ|カフェ|外|
|の|助詞|格助詞|*|の|ノ|ノ|和|
|違い|名詞|普通名詞|一般|違い|チガイ|チガイ|和|
|は|助詞|係助詞|*|は|ワ|ハ|和|
|意外|形状詞|一般|*|意外|イガイ|イガイ|漢|
|と|助詞|格助詞|*|と|ト|ト|和|
|明確|形状詞|一般|*|明確|メーカク|メイカク|漢|


This table is an example of the output available from fugashi and UniDic. Note how besides tokenization it includes a variety of information about each token. This is only some of the fields available in UniDic. :en

以上の表は fugashi と UniDic の出力の例です。単語分割のみならず、各単語について様々な情報も含まれています。以上で表示されている情報もまた UniDic の情報の一部でしかありません。 :ja

### Setup :en

### 事前準備 :ja

First you'll need to install fugashi and the dictionary.  :en

まず fugashi とその辞書をインストールする必要があります。 :ja

**fugashi** is a wrapper for **MeCab**, a classic Japanese morphological analyzer. fugashi uses Cython to access MeCab's C interface, and also includes some convenient tweaks to make it easier to use in Python. :en

**fugashi** は昔からある、人気の形態素解析器 **MeCab** のラッパー[^wrapper_ja]です。Cython を用いて MeCab の C API を Python から使えるようにした上に、Python から便利に使えるように細かい変更が加えられています。 :ja

[wrapper_ja]: 「ラッパー」(英：wrapper, 包み)とは、あるプログラムを「包んで」別の API を提供するプログラムやライブラリのことです。

**unidic-lite** is a slightly modified version of UniDic 2.1.2. That version of UniDic is somewhat old, but it's small enough that it's easy to install, and high quality enough that it's sufficient for most applications. The **unidic** package on PyPI wraps the latest edition of UniDic, but due to a large increase in dictionary entries, it's harder to set up, so we won't use it for this tutorial. :en

**unidic-lite**は、UniDic 2.1.2 をベースに変更を加えた形態素解析用の辞書です。UniDic のバージョンとしてはやや古いですが、情報の質に問題はなく、後のバージョンと比べてデータ量も少なく、扱いやすいという特徴があります。PyPI の **unidic** パッケージを使えば最新版の UniDic も利用できますが、辞書の見出し語数が大量に増えたせいで環境構築が少しばかり複雑なので、本チュートリアルでは使用しません。:ja

At time of writing the latest version of fugashi is 1.1.0 and the latest version of unidic-lite is 1.0.8. unidic-lite will work on any system, and fugashi distributes ready-to-use "wheels" for OSX, Linux, and 64 bit Windows. (If you have another operating system you may have to build from source. If you have trouble please feel free to [open an issue](https://github.com/polm/fugashi).) :en

本書執筆時での fugashi の最新版は 1.1.0、unidic-lite は 1.0.8 です。unidic-lite は純粋にデータのみから構成されており、どの環境でも利用できます。fugashi は OSX・Linux・Win64 の「wheel」を提供していますので、そのいずれの環境であれば他の事前準備は不要です。(他の環境ではソースからビルドする必要があります。もしビルド中に何か問題がありましたら、お気軽に[issue を立ててください](https://github.com/polm/fugashi)。） :ja

In [1]:
%%capture
!pip install fugashi unidic-lite

Now that fugashi is installed, you can confirm it works by running it in the terminal. Try running `fugashi -O wakati` and then typing some Japanese. If you push Enter, your input text will be printed with spaces separating tokens. You can use `CTRL+D` to terminate the process. Here's some example output: :en

これで fugashi がインストールされましたので、正しくインストールされているか確認するために一度シェルから実行してみましょう。まずは、`fugashi -O wakati` を実行し適当な文章を入力してみましょう。文章を入力した後に改行を入力すると、入力文がスペース区切りで返ってきます。出力の確認が終わったら `CTRL+D` でプログラムを終了できます。出力は以下のようになります。 :ja

In [2]:
!echo "毎年東麻布ではかかし祭りが開催されます" | fugashi -O wakati

毎年 東 麻布 で は かかし 祭り が 開催 さ れ ます


Note: `wakati` comes from 分かち書き *wakachigaki*, which refers to the practice of writing Japanese with spaces included, as used in children's books and low resolution displays. In MeCab this refers to the special output mode that just separates tokens with spaces. Note that real wakachigaki uses spaces to separate bunsetsu, not tokens or words. :en

ここで使ったオプション `wakati` は「分かち書き」のことを指しています。「分かち書き」とは、子供向けの本や、低解像度の画面ゆえにひらがなのみで書かれた文章などで見られる、スペース区切りで書かれた日本語の文章のことです。本来、分かち書きは文節をスペースで区切るのが普通ですが、MeCab での分かち書きは単語 (形態素) 単位で文章を区切ります。 :ja

Next let's use fugashi in code. The main interface to the library is the `Tagger` object, which holds a variety of dictionary related state. The primary way to use the `Tagger` is to simply apply it to input text, which will return a list of `Node` objects. Each `Node` contains the raw text of the token in a `surface` property, and extended dictionary fields are available in the `feature` property. :en

それではコードから fugashi を使ってみましょう。コードでは主に辞書の情報を管理する `Tagger` オブジェクトを使います。 `Tagger` を入力分に適応すると `Node` のリストが返ってきます。 `Node` のテキストは `surface` 属性に格納され、辞書のさまざまな情報は `feature` 属性に格納されています。 :ja

In [3]:
import fugashi

tagger = fugashi.Tagger()

text = "形態素解析をやってみた"
words = tagger(text)
print(words)
print("=====")

for word in words:
    print(word.surface, word.feature.lemma, word.feature.kana, sep="\t")

[形態, 素, 解析, を, やっ, て, み, た]
=====
形態	形態	ケイタイ
素	素	ソ
解析	解析	カイセキ
を	を	ヲ
やっ	遣る	ヤッ
て	て	テ
み	見る	ミ
た	た	タ


Note: In Japanese NLP, it's standard to refer to the raw input text form as the "surface" (表層 *hyousou*), and MeCab uses this in its API. This usage comes from linguistics, where the **surface form** of a word in a particular context (which may be inflected or have unusual orthography) is contrasted with the **lexical form**, which would be a normalized or dictionary form. :en

日本語の言語処理では、原文の表記そのままのことを指して「表層」（英：surface）と呼ぶことが多いです。これはもともと言語学の用語で、語形変化や表記の違いなども含めた、原文のままの「表層形式」の他に、辞書の見出しなどで使われる、標準化した「語彙形式」（英語：lexical form）があります。 :ja

For basic tokenization, this is all you need to know. In the next section, we'll look at a slightly more involved application of morphological analysis, and later in this chapter we'll cover advanced tokenization-related topics. :en

基本的な単語分割の処理の紹介は以上です。次の節では、もう少し高度な形態素解析の応用を紹介し、その後、更に高度な使い方などを説明します。 :ja

### Morphological Analysis Mini Project: Automatic Fuseji :en

### 形態素解析を使ってみよう　自動伏せ字プログラムの実装 :ja

*Fuseji* (伏せ字) is the practice of replacing some characters with placeholders, usually a circle, to conceal the content of words. A similar thing is sometimes done in English, particularly to avoid using obscene words (`a**hole`, "you little @#%(*!"). In Japanese fuseji can be used for obscene words, but they can also be used to avoid spoilers, be vague about the names of brands or specific people, or for other reasons. :en

**伏せ字**とは、文章の一部の文字を、他の文字に置き換え、単語の一部を隠すことを指します。個人名や商品名の明言を避けるため、ネタバレを避けるため、検索避けのため、など、様々な目的に使われます。 :ja

Let's pretend that we want to automatically apply fuseji for the purpose of hiding spoilers about new movies or other media. While the simplest thing is to replace characters at random from the whole string, it's better to replace certain kinds of words, such as proper nouns. We can use the detailed part of speech information in UniDic, along with word boundaries, to replace proper nouns with fuseji versions. :en

今回は、映画などの新作作品のネタバレを避けるために、自動的に伏せ字を適用するプログラムを書きたいとしましょう。単に入力文章の一部の文字を伏せたいだけであれば、文字をランダムに置換することもできますが、品詞などに基づいてどこを伏せるかを決めると、結果がより自然になります。UniDic の細かい品詞情報と単語分割の結果を活用し、まずは固有名詞を伏せてみましょう。 :ja

In [4]:
from fugashi import Tagger
from random import sample

tagger = Tagger()


def fuseji_node(text, ratio=1.0):
    """This function will take a node from tokenization and actually replace parts of it with filler characters.:en"""
    """分かち書きの結果ノードを受け取り、文字列の一部をランダムに◯で置き換える :ja"""
    ll = len(text)
    idxs = sample(range(ll), max(1, int(ratio * ll)))
    out = []
    for ii, cc in enumerate(text):
        out.append("◯" if ii in idxs else cc)
    return "".join(out)


def fuseji_text(text, ratio=1.0):
    """Given an input string, apply fuseji. :en"""
    """入力文を受け取り、適切なところを伏せ字に置き換える :ja"""
    out = []
    for node in tagger(text):
        # Normal Japanese text doesn't use white space, but this is necessary :en
        # if you include latin text, for example. :en
        # 純粋な日本語のテキストはスペースを含まないが、英語のテキストと混ぜる場合などに必要になる。 :ja
        # スペースはそれ自体でノードにならないので、ここで明示的に追加する必要がある。 :ja
        out.append(node.white_space)
        if node.feature.pos2 != "固有名詞":
            out.append(node.surface)
        else:
            out.append(fuseji_node(node.surface))
    return "".join(out)


print(fuseji_text("犯人はヤス"))
print(fuseji_text("東京タワーの高さは333m"))

犯人は◯◯
◯◯タワーの高さは333m


This code is already reasonably effective, but there are several ways it could be tweaked or improved. For example, sometimes the words that should be concealed aren't just proper nouns; they could also be ordinary nouns or verbs. :en

このプログラムはこのままでもそれなりに使えますが、改善の余地はまだ残っています。場合によっては固有名詞だけでなく、一般名詞や動詞を伏せたいときもあります。 :ja

How can we find what parts of speech we want to filter? The best way is to use **example sentences** to find what parts of speech we want, as well as to get a better understanding of where our program works well and where it doesn't. :en

伏せたい品詞はどうやって特定すれば良いでしょうか。品詞表から調べられることも可能ですが、例文をいくつか実際に解析してみて、その結果を見る方が楽で効果的です。実際データを処理してみると、どんなケースで伏せ字プログラムがうまくいくかどうかがより深く理解できます。 :ja

This list is the same in both languages :xx
- 新キャラの「カズヤ」は年内に配信予定
- マジルテの水晶の畑エリアにはクリスタルが沢山ある
- 「吾輩は猫である」の作家は夏目漱石
- 『さかしま』（仏: À rebours）は、フランスの作家ジョリス＝カルル・ユイスマンスによる小説

We can check the parts of speech of words in fugashi by using the `node.pos` attribute. This part of speech information comes from UniDic and uses four levels. You can access the individual levels as `node.feature.pos1`, `node.feature.pos2`, and so on. The `node.pos` attribute is a convenience feature that joins the four separate values together and replaces empty values with an asterisk (`*`). :en

fugashi の解析結果の品詞は、属性 `node.pos`[^pos_ryaku] を見ることによって確認できます。この品詞体系は UniDic のものに基づいており、４つのレベルから構成されます。各レベルは、`node.feature.pos1`, `node.feature.pos2`, ... を使って参照できます。`node.pos`は4つ全てのレベルの情報を一つの文字列にまとめた属性です。全てのレベルに情報があるとは限らず、情報がない場合、そのレベルは `*`になります。 :ja

[^pos_ryaku]: `pos`は「part of speech (品詞）」の略です。

You can check part of speech tags of words by giving a sentence as input with fugashi on the command line, without giving the `-O wakati` command line argument. :en

例文の品詞タグはシェルから確認できます。上で試したのとは異なり、`-O wakati` を指定しない場合、各行ごとに、品詞情報などを含む単語の情報が表示されます。 :ja

In [5]:
!echo "毎年東麻布ではかかし祭りが開催されます" | fugashi

毎年	マイトシ	マイトシ	毎年	名詞-普通名詞-副詞可能			0
東	ヒガシ	ヒガシ	東	名詞-普通名詞-一般			0,3
麻布	アザブ	アザブ	アザブ	名詞-固有名詞-地名-一般			0
で	デ	デ	で	助詞-格助詞			
は	ワ	ハ	は	助詞-係助詞			
かかし	カカシ	カカス	欠かす	動詞-一般	五段-サ行	連用形-一般	0,2
祭り	マツリ	マツリ	祭り	名詞-普通名詞-一般			0
が	ガ	ガ	が	助詞-格助詞			
開催	カイサイ	カイサイ	開催	名詞-普通名詞-サ変可能			0
さ	サ	スル	為る	動詞-非自立可能	サ行変格	未然形-サ	0
れ	レ	レル	れる	助動詞	助動詞-レル	連用形-一般	
ます	マス	マス	ます	助動詞	助動詞-マス	終止形-一般	
EOS


### Censoring Unknown Words :en
### 未知語を伏せ字に変換する :ja

Another thing that'll come up as we're testing is that sometimes words not in the dictionary will be used, like the names of characters in movies and books. From the example sentences above, マジルテ *Majirute*, the name of a fictional place, is an example of such a word. We basically always want to censor those words to avoid spoilers, so rather than checking part of speech information, we can also check specifically for words that aren't in our dictionary. These are called "unks", from "unknown words", or 未知語 *michigo* in Japanese. In fugashi you can determine if a given node is in the dictionary just by checking the `node.is_unk` attribute. :en

様々な文を使って形態素解析を試してみると、映画や書籍のキャラクターの名前など、辞書に含まれていない単語に出くわします。上の例文中の「マジルテ」(架空の場所の名前) もその一つです。ネタバレを避けるために、これらの単語も積極的に伏せたいので、この場合品詞ではなく「辞書にないこと」に基づいて伏せ字の対象にします。これら「辞書にない単語」は「未知語」、英語では「unk」(unknownの略)と呼ばれます。fugashi では`node.is_unk` の属性を見ることで、その単語が未知語かどうかが確認できます。 :ja

Looking at our example sentences, some patterns emerge. We probably don't want to filter verbs, since it's hard to tell when a verb is important. Proper nouns should definitely be filtered. Common nouns may or may not be important, so it's hard to say if we should filter them - for now, let's leave them alone. :en

これらの例文を確認していくと、該当する品詞が分かってきます。動詞については、重要なものとそうでないものを品詞だけでは区別することができないので、除外しない方が良いでしょう。逆に、固有名詞は全部伏せ字対象にした方が良さそうです。一般名詞は動詞と同様、伏せ字にしたいものとそうでないものがあるので、とりあえず対象外にしましょう。 :ja

Since our conditions for censoring words are getting kind of complicated, let's factor them into a function. :en

伏せ字対象の条件がやや複雑化しているので、ここで一旦関数にまとめましょう。:ja

In [6]:
def should_hide(node):
    """Check if this node should be hidden or not. :en"""
    """与えられたノードを伏せ字の対象にするかどうかを返す :ja"""
    if node.is_unk:
        return True
    ff = node.feature
    if ff.pos1 == "名詞" and ff.pos2 == "固有名詞":
        return True
    return False


def fuseji_text(text, ratio=1.0):
    """Given an input string, apply fuseji. :en"""
    """与えられた文字列に対して、伏せ字を適用する :ja"""
    out = []
    for node in tagger(text):
        out.append(node.white_space)
        word = fuseji_node(node.surface) if should_hide(node) else node.surface
        out.append(word)
    return "".join(out)


texts = [
    "犯人はヤス",
    "魔法の言葉はヒラケゴマ",
    "『さかしま』（仏: À rebours）は、フランスの作家ジョリス＝カルル・ユイスマンスによる小説",
    "鈴木爆発で最初に解体する爆弾はみかんの形をしている",
]

for text in texts:
    print(fuseji_text(text))

犯人は◯◯
魔法の言葉は◯◯◯◯◯
『さかしま』（仏: ◯ ◯◯◯◯◯◯◯）は、◯◯◯◯の作家◯◯◯◯＝◯◯◯・◯◯◯◯◯◯による小説
◯◯爆発で最初に解体する爆弾はみかんの形をしている


### Use Readings to Censor only Part of Words :en
### 読みを使って単語の一部を伏せ字にする :ja

At this point our program is pretty effective at applying fuseji to any text we throw at it. That said, censoring the entire text is a little boring. It would be more interesting if we could reveal some letters so that readers can guess the rest of the word, but not quite be certain about it. :en

この時点で、このプログラムはどんな文章でもそれなりに伏せ字を適用することができます。ただし、伏せ字対象となった単語の全ての文字をただ単に伏せるのは少し味気が無いですね。一部の文字だけを伏せ、読み手が単語の残りを少しだけ推測できるようにすればさらに面白いと思われます。:ja

There is one potential issue though - if we use kanji, even one character might give the word away in away that's not interesting. What if we could convert words to phonetic versions and *then* censor part of them? That would allow us to show part of the word while giving away less information. :en

しかし、一部の文字だけを伏せると問題が発生します。漢字の単語だと、文字一つだけで元の単語が分かってしまうこともあり、これも面白くありません。これを回避するために、まず単語を読み仮名に変換し、その読みの一部を伏せればどうでしょう。そうすれば、一部の文字を伏せなくても、そう簡単に原文がバレることはないはずです。 :ja

Thankfully, UniDic includes a field we can use for this conversion. Every word in the UniDic dictionary has a `kana` field we can use to get the conventional reading for the word in katakana form. (UniDic also has a `pron` field, which uses non-standard orthography to differentiate long vowels.) :en

UniDic には読み情報もちゃんと収録されているので、この変換に使うことができます。UniDic では、全ての項目に対して `kana` フィールドがあり、表層をこれに置き換えることによって読み仮名に変換できます。（これとは別に `pron` フィールドもありますが、これは一般的な読み仮名ではなく、棒引き仮名遣いを使った表記のものです。）:ja

One thing to keep in mind is that the kana reading will only be available for words in UniDic, and it won't always be perfect. There are two cases where the reading will be wrong: :en

注意点として、読み情報は UniDic に収録されいる単語にしか使えないことと、収録されている単語であっても、読み方が文脈に照らし合わせていつも正しいとは限らないことがあります。読み仮名が正しくない場合は大きく分けて2つあります。 :ja

1. The word is not in the dictionary. :en
2. The reading of the word is ambiguous. :en

1. 未知語の場合 :ja
2. 読み方が曖昧な単語 :ja

If the word is not in the dictionary, it's possible to train a machine learning model or use other methods to predict the reading, but that's pretty difficult. So this time, if a word is an unk we'll just skip converting it and use the raw surface form. :en

未知語の場合、機械学習などを使って読み仮名を判定するプログラムを書くことは可能ですが、簡単ではありません。したがって今回は、未知語はあきらめて表層をそのまま使います。 :ja

Ambiguous words are more difficult. Some examples of ambiguous words: :en

同形異音語（形は同じでも読み方が曖昧な単語）は未知語よりも対応が難しいです。同形異音語の例には以下のようなものがあります。 :ja

:en

- 東: *higashi* or *azuma* (or *tou*)
- 中田: *nakada* or *nakata*
- 仮名: *kana* or *kamei*
- 網代: *amishiro* or *ajiro*
- 最中: *saichuu* or *monaka*
- 私: *watashi* or *watakushi*
- 日本: *nihon* or *nippon*

:end

:ja

- 東: ひがし、あずま、とう
- 中田: なかだ、なかた
- 仮名: かな、かめい
- 牧場: ぼくじょう、まきば
- 網代: あみしろ、あじろ
- 日本: にほん、にっぽん

:end

Usually a reading will be clear from context, but many ambiguous words are proper nouns like the names of people and places, and without knowing which specific entity it's referring to there's no way to be sure of the correct reading. Even worse, there's no way to be sure if the word you're looking at is ambiguous or not just using the tokenizer output. :en

文脈から読み方が分かる場合も多いですが、同形異音語の多くは人名や地名などの固有名詞なので、どの人・場所に対して使われているかを知らないと、読みが判別できないこともあります。更に大変なことに、MeCab の出力を見るだけでは、ある単語の読みが曖昧かどうかは分かりません。 :ja

(Note: Words written the same way but pronounced differently are referred to as 同形異音語 *doukei iongo* or "heteronyms". They are also common in English, though less so for proper nouns.) :en

Note: For ambiguous words, deciding their reading could be considered a form of **word sense disambiguation** for common nouns, or **entity linking** for proper nouns. Both are NLP problems with a long history. :en

同形異音語の読みの判別は、一般名詞の場合は**語義の曖昧性解消** (word sense disambiguation)、固有名詞の場合は**エンティティ・リンキング** (entity linking)のタスクであると捉えることもできます。これらのタスクは自然言語処理の分野では昔から多くの研究が行われてきた課題です。 :ja

So how can we handle ambiguous words if we can't even identify them with certainty? It turns out that their difficulty actually has a silver lining - because even people make mistakes, we can get away with just using the kana UniDic gives us and hope that it's right most of the time. For serious applications replacing the original text with a mistake would be unacceptable, but for our fuseji application, it's not the end of the world if we're wrong occasionally.  :en

同形異音語の読みがきちんと判定できないのであれば、どう処理すれば良いでしょう。実はその曖昧性自体がある種の救いになっています。読み方が曖昧な単語だと、人間でも読みを間違えることがあるので、プログラムが間違った読みを出しても読み手がある程度分かってくれます。読み間違いが許されないような応用も数多くありますが、今回の伏せ字プログラムでは、これはあまり気にしなくても特に問題にはなりません。 :ja

Sometimes when you learn about a problem confronting your NLP system, there may not be a solution you're able to implement. In this case, writing a program to disambiguate words would be much more work than the rest of our entire program. But by being aware of the problem, we can consider how failures affect the output of our system, and evaluate whether we should continue with its development, or start over with a design that can work around the problem. :en

NLP システムの開発中に問題に出くわした場合、その問題の解決方法を実装するのが現実的ではない場合があります。例えばこの伏せ字プログラムの場合、同形異音語の読みの曖昧性を解消するプログラムを書くことは可能ですが、おそらく解決に必要な作業量は、伏せ字プログラムの他の部分を上回ってしまうでしょう。ただし、問題を正しく認識していれば、うまくいかない場合が出力にどういう影響を及ぼすかを考え、元のプログラムを改善する方法を閃くこともあります。場合によっては開発を断念せざるを得ないこともありますが、逆に問題を最初から回避する仕組み導入できる場合もあります。:ja

Now that we've settled that, let's change our code to use the kana instead of the surface when censoring words. :en

それでは早速、読み仮名に変換するところを実装しましょう。 :ja

In [7]:
def fuseji_text(text, ratio=1.0):
    """Given an input string, apply fuseji. :en"""
    """入力文に伏せ字を適用する :ja"""
    out = []
    for node in tagger(text):
        out.append(node.white_space)
        node_text = node.surface if node.is_unk else node.feature.kana
        word = fuseji_node(node_text, ratio=0.5) if should_hide(node) else node.surface
        out.append(word)
    return "".join(out)


texts = [
    "黒幕の正体はガーランド",
]

for text in texts:
    print(fuseji_text(text))

黒幕の正体は◯ーラ◯ド


And that makes our automatic fuseji program complete. It's not a lot of code, but in building this you learned how to: :en

これで今回の伏せ字プログラムは完成となります。行数は決して多くはありませんが、これを書く過程で、下記の機能の使い方を紹介しました。:ja

:en

1. iterate over the tokens in a text
2. identify parts of speech of interest with example sentences
3. use multiple levels of part of speech tags
4. check if a token is in the dictionary or an unk
5. convert words to their phonetic representation

:end

:ja

1. 文章の単語を一つずつ処理する方法
2. 例文を使って目的の品詞を特定する方法
3. 品詞の構造の扱い
4. 未知語の判別
5. 読み仮名変換

:end

These are all basic building blocks you can use to build a wide variety of applications. :en

これらの機能はどちらも基礎的な処理なので、組み合わせによってさまざまなプログラムを作ることができます。 :ja

While our motivation for this program was a simple and playful one, the techniques used here are simple versions of those used in **personally identifying information (PII) removal**, which removes identifying details from documents like medical and legal records so they can be used in audits or analysis without risk to the people they describe. :en

今回の伏せ字プログラムは単純で遊び的なものに過ぎませんが、ここで使った技術をさらに発展させ、実用的な個人情報漏洩防止ツールの実装も可能です。このようなツールは、カルテのような医療情報や契約書などの法的文書を監査や解析に出す前に、個人情報を特定・削除するために広く使われています。:ja

To learn more about the tokenizer API, consider some ways you might want to extend this application and how you'd make the necessary changes. :en

MeCab の API を更に深く理解するために、下記の場合、どうやってこの伏せ字プログラムを変更するか考えてみましょう。 :ja

:en

- what if you wanted to remove all numbers from a contract, to hide dates or prices?
- what if you wanted to hide a specific list of words, perhaps obscenities, rather than certain parts of speech?
- how would you change the program to replace hard-to-read words with their phonetic versions?

:end

:ja

- 契約書から日付や金額などの数字を消す
- 品詞によってではなく、禁止語など特定の単語を伏せる
- 難読語を読み仮名に変換する

:end